<a href="https://colab.research.google.com/github/JumpNShootMan/DataMining_MarketAnalysis/blob/main/PC2_Data_Mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PC2 Data Mining
## By: Daniel Núñez Robinson

### Import .zip files from drive and extract the datasets

In [ ]:
from google.colab import drive
import zipfile
import pandas as pd
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Important...
The dataset path needs to be set manually. I couldn't find a way to upload it to GitHub since the size was too big on the order dataset.
https://drive.google.com/drive/folders/1SRMY1hM3fFPyW88L1lLDVwCiqvlEMuAG?usp=sharing

In [ ]:
base = "/content/drive/My Drive/2020-2/Data_Mining/PC2/"

zipProducts = zipfile.ZipFile("/content/drive/My Drive/2020-2/Data_Mining/PC2/products.csv.zip","r")
zipProducts.extractall()

zipPrior = zipfile.ZipFile("/content/drive/My Drive/2020-2/Data_Mining/PC2/order_products__prior.csv.zip","r")
zipPrior.extractall()

In [ ]:
dataProd = pd.read_csv("products.csv")
dataOrder = pd.read_csv("order_products__prior.csv")

### Read the datasets

In [ ]:
dataProd.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [ ]:
dataOrder.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


## Replicate original author's work

In [ ]:
import numpy as np
from itertools import combinations, groupby
from collections import Counter

# Sample data
order = np.array([[1,'apple'], [1,'egg'], [1,'milk'], [2,'egg'], [2,'milk']], dtype=object)

# Generator that yields item pairs, one at a time
def get_item_pairs(order_item):
    
    # For each order, generate a list of items in that order
    for order_id, order_object in groupby(dataOrder, lambda x: x[0]):
        item_list = [item[1] for item in order_object]      
    
        # For each item list, generate item pairs, one at a time
        for item_pair in combinations(item_list, 2):
            yield item_pair                                      


# Counter iterates through the item pairs returned by our generator and keeps a tally of their occurrence
Counter(get_item_pairs(dataOrder))

Counter({('apple', 'egg'): 1, ('apple', 'milk'): 1, ('egg', 'milk'): 2})

In [ ]:
import pandas as pd
import numpy as np
import sys
from itertools import combinations, groupby
from collections import Counter
from IPython.display import display

#### Order data

In [ ]:
# Function that returns the size of an object in MB
def size(obj):
    return "{0:.2f} MB".format(sys.getsizeof(obj) / (1000 * 1000))

In [ ]:
print('dataOrder -- dimensions: {0};   size: {1}'.format(dataOrder.shape, size(dataOrder)))
display(dataOrder.head())

dataOrder -- dimensions: (32434489, 4);   size: 1037.90 MB


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


#### Change format

In [ ]:
# Convert from DataFrame to a Series, with order_id as index and item_id as value
dataOrder = dataOrder.set_index('order_id')['product_id'].rename('item_id')
display(dataOrder.head(10))
type(dataOrder)

order_id
2    33120
2    28985
2     9327
2    45918
2    30035
2    17794
2    40141
2     1819
2    43668
3    33754
Name: item_id, dtype: int64

pandas.core.series.Series

In [ ]:
print('dimensions: {0};   size: {1};   unique_orders: {2};   unique_items: {3}'
      .format(dataOrder.shape, size(dataOrder), len(dataOrder.index.unique()), len(dataOrder.value_counts())))

dimensions: (32434489,);   size: 518.95 MB;   unique_orders: 3214874;   unique_items: 49677


#### Association Rules Function

##### Helper Function

In [ ]:
# Returns frequency counts for items and item pairs
def freq(iterable):
    if type(iterable) == pd.core.series.Series:
        return iterable.value_counts().rename("freq")
    else: 
        return pd.Series(Counter(iterable)).rename("freq")

    
# Returns number of unique orders
def order_count(order_item):
    return len(set(order_item.index))


# Returns generator that yields item pairs, one at a time
def get_item_pairs(order_item):
    order_item = order_item.reset_index().values
    for order_id, order_object in groupby(order_item, lambda x: x[0]):
        item_list = [item[1] for item in order_object]
              
        for item_pair in combinations(item_list, 2):
            yield item_pair
            

# Returns frequency and support associated with item
def merge_item_stats(item_pairs, item_stats):
    return (item_pairs
                .merge(item_stats.rename(columns={'freq': 'freqA', 'support': 'supportA'}), left_on='item_A', right_index=True)
                .merge(item_stats.rename(columns={'freq': 'freqB', 'support': 'supportB'}), left_on='item_B', right_index=True))


# Returns name associated with item
def merge_item_name(rules, item_name):
    columns = ['itemA','itemB','freqAB','supportAB','freqA','supportA','freqB','supportB', 
               'confidenceAtoB','confidenceBtoA','lift']
    rules = (rules
                .merge(item_name.rename(columns={'item_name': 'itemA'}), left_on='item_A', right_on='item_id')
                .merge(item_name.rename(columns={'item_name': 'itemB'}), left_on='item_B', right_on='item_id'))
    return rules[columns]  

##### Association rules function

In [ ]:
def association_rules(order_item, min_support):

    print("Starting order_item: {:22d}".format(len(order_item)))


    # Calculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) * 100


    # Filter from order_item items below min support 
    qualifying_items       = item_stats[item_stats['support'] >= min_support].index
    order_item             = order_item[order_item.isin(qualifying_items)]

    print("Items with support >= {}: {:15d}".format(min_support, len(qualifying_items)))
    print("Remaining order_item: {:21d}".format(len(order_item)))


    # Filter from order_item orders with less than 2 items
    order_size             = freq(order_item.index)
    qualifying_orders      = order_size[order_size >= 2].index
    order_item             = order_item[order_item.index.isin(qualifying_orders)]

    print("Remaining orders with 2+ items: {:11d}".format(len(qualifying_orders)))
    print("Remaining order_item: {:21d}".format(len(order_item)))


    # Recalculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) * 100


    # Get item pairs generator
    item_pair_gen          = get_item_pairs(order_item)


    # Calculate item pair frequency and support
    item_pairs              = freq(item_pair_gen).to_frame("freqAB")
    item_pairs['supportAB'] = item_pairs['freqAB'] / len(qualifying_orders) * 100

    print("Item pairs: {:31d}".format(len(item_pairs)))


    # Filter from item_pairs those below min support
    item_pairs              = item_pairs[item_pairs['supportAB'] >= min_support]

    print("Item pairs with support >= {}: {:10d}\n".format(min_support, len(item_pairs)))


    # Create table of association rules and compute relevant metrics
    item_pairs = item_pairs.reset_index().rename(columns={'level_0': 'item_A', 'level_1': 'item_B'})
    item_pairs = merge_item_stats(item_pairs, item_stats)
    
    item_pairs['confidenceAtoB'] = item_pairs['supportAB'] / item_pairs['supportA']
    item_pairs['confidenceBtoA'] = item_pairs['supportAB'] / item_pairs['supportB']
    item_pairs['lift']           = item_pairs['supportAB'] / (item_pairs['supportA'] * item_pairs['supportB'])
    
    
    # Return association rules sorted by lift in descending order
    return item_pairs.sort_values('lift', ascending=False)

#### Association Rules Mining

In [ ]:
#%%time
rules = association_rules(dataOrder, 0.01) 

Starting order_item:               32434489
Items with support >= 0.01:           10906
Remaining order_item:              29843570
Remaining orders with 2+ items:     3013325
Remaining order_item:              29662716
Item pairs:                        30622410
Item pairs with support >= 0.01:      48751



In [ ]:
# Replace item ID with item name and display association rules
item_name   = pd.read_csv('products.csv')
item_name   = item_name.rename(columns={'product_id':'item_id', 'product_name':'item_name'})
rules_final = merge_item_name(rules, item_name).sort_values('lift', ascending=False)
display(rules_final)

,itemA,itemB,freqAB,supportAB,freqA,supportA,freqB,supportB,confidenceAtoB,confidenceBtoA,lift
0,Organic Strawberry Chia Lowfat 2% Cottage Cheese,Organic Cottage Cheese Blueberry Acai Chia,306,0.010155,1163,0.038595,839,0.027843,0.263113,0.364720,9.449868
1,Grain Free Chicken Formula Cat Food,Grain Free Turkey Formula Cat Food,318,0.010553,1809,0.060033,879,0.029170,0.175788,0.361775,6.026229
3,Organic Fruit Yogurt Smoothie Mixed Berry,Apple Blueberry Fruit Yogurt Smoothie,349,0.011582,1518,0.050376,1249,0.041449,0.229908,0.279424,5.546732
9,Nonfat Strawberry With Fruit On The Bottom Gre...,"0% Greek, Blueberry on the Bottom Yogurt",409,0.013573,1666,0.055288,1391,0.046162,0.245498,0.294033,5.318230
10,Organic Grapefruit Ginger Sparkling Yerba Mate,Cranberry Pomegranate Sparkling Yerba Mate,351,0.011648,1731,0.057445,1149,0.038131,0.202773,0.305483,5.317849
...,...,...,...,...,...,...,...,...,...,...,...
7271,Organic Strawberries,Strawberries,640,0.021239,263416,8.741706,141805,4.705931,0.002430,0.004513,0.000516
6763,Organic Hass Avocado,Organic Avocado,464,0.015398,212785,7.061469,176241,5.848722,0.002181,0.002633,0.000373
4387,Organic Avocado,Organic Hass Avocado,443,0.014701,176241,5.848722,212785,7.061469,0.002514,0.002082,0.000356
2596,Banana,Bag of Organic Bananas,654,0.021704,470096,15.600574,376367,12.490090,0.001391,0.001738,0.000111


## What does this code do?

The code begins by calculating the frequency and support for all of theitem pairs generated in the previous cells.  

Next, the item pairs frequencies and supports are calculated. Afterwards, the data is filtered to only show orders above the minimum support and at least 2 items.  

Finally, a table is created with the association rules displayed, including the frequencies, support calculations and lift values.

Calculated values:


*   Confidence: Given items A and B, measures the percentage of times that item B is purchased when A was purchased.
*   Support: The percentage of orders that contain the item set
*   Lift: Given the 2 items A and B, this indicates wether there is a relationship between both items or if they are in the same orders by chance.



## Is it possible to mine emerging patterns?

It is possible to mine emerging patterns by creating a subset of the orders and evaluating the frequency of the entire dataset of patterns to find any insights.

## Is it possible to mine sky patterns?
It is possible to mine patterns by evaluating the patterns with the biggest frecuencies and  highest confidence or lift values, which are calculations based on the confidence values and whose values > 1 indicate a positive relationship between the patterns.